<div style="text-align: center;">
    <h1> <strong>Nom et Prénom : Laakel Gauzi Soumaya</strong> </h1>
    <h3><strong>Module : NATURAL LANGUAGE PROCESSING (NLP)</strong></h3>
    <h4><em>MST IASD/S1 2024-2025</em></h4>
    <hr>
    <h2><strong>DEVOIR 4 : PRATIQUER L’ANALYSE SYNTAXIQUE</strong></h2>
    <p><i>L’objectif de ce devoir est d’utiliser la bibliothèque spaCy pour analyser une phrase en français, en identifiant automatiquement les catégories grammaticales et les relations de dépendance entre les mots. Il s’agit notamment d’extraire les éléments clés de la phrase, comme le verbe principal, le sujet, l’objet direct et les compléments circonstanciels, afin de comprendre la structure syntaxique et le sens de la phrase.</i></p>
</div>

# Étape 1 : Charger le texte avec spaCy

In [3]:
import spacy

# Charger le modèle français
nlp = spacy.load("fr_core_news_sm")

# Texte donné
texte = "Le médecin prescrit des antibiotiques au patient fiévreux après le diagnostic."

# Traitement du texte
doc = nlp(texte)

# Étape 2 : Afficher POS et dépendances pour chaque mot

Pour chaque mot (token), on va affiche :
- Le texte du mot.
- Sa catégorie grammaticale (POS : Part of Speech).
- Sa relation de dépendance syntaxique dans la phrase.

In [5]:
for token in doc:
    print(f"{token.text:<15} POS: {token.pos_:<10} Dépendance: {token.dep_}")

Le              POS: DET        Dépendance: det
médecin         POS: NOUN       Dépendance: nsubj
prescrit        POS: ADJ        Dépendance: ROOT
des             POS: ADP        Dépendance: case
antibiotiques   POS: NOUN       Dépendance: obl:arg
au              POS: ADP        Dépendance: case
patient         POS: NOUN       Dépendance: obl:mod
fiévreux        POS: ADJ        Dépendance: acl
après           POS: ADP        Dépendance: case
le              POS: DET        Dépendance: det
diagnostic      POS: NOUN       Dépendance: nmod
.               POS: PUNCT      Dépendance: punct


## Interprétation de l'analyse syntaxique

* **Le** → Déterminant (*DET*), dépend du mot « médecin » (fonction : déterminant du nom).
* **médecin** → Nom (*NOUN*), c’est le **sujet** principal de la phrase (*Dépendance : nsubj*).
* **prescrit** → Marqué comme adjectif (*ADJ*) ici, mais en réalité c’est le **verbe** principal, le **noyau** de la phrase (*Dépendance : ROOT*).
* **des** → Préposition (*ADP*), introduit le complément « des antibiotiques » (*case*).
* **antibiotiques** → Nom (*NOUN*), c’est le **complément d’objet indirect** du verbe (*Dépendance : obl\:arg*).
* **au** → Préposition (*ADP*), introduit le complément « au patient » (*case*).
* **patient** → Nom (*NOUN*), c’est un **complément circonstanciel** du verbe (*Dépendance : obl\:mod*).
* **fiévreux** → Adjectif (*ADJ*), qualifie le « patient » (*Dépendance : acl*).
* **après** → Préposition (*ADP*), introduit une circonstance de temps (*case*).
* **le** → Déterminant (*DET*), dépend de « diagnostic » (*det*).
* **diagnostic** → Nom (*NOUN*), c’est le complément introduit par « après » (*Dépendance : nmod*).
* **.** → Ponctuation (*PUNCT*), marque la fin de la phrase (*punct*).


###  Remarque!!!
Le modèle spaCy a classé **"prescrit"** comme un adjectif (ADJ), ce qui est une petite erreur. En contexte, c'est un verbe (il décrit l'action principale). Ce genre de petite imprécision est normal dans le TAL.

# Étape 3 : Trouver les éléments clés

## 3.a Le verbe principal

In [6]:
for token in doc:
    if token.dep_ == "ROOT":
        print("Verbe principal :", token.text)

Verbe principal : prescrit


## 3.b Son sujet

In [7]:
for token in doc:
    if token.dep_ == "nsubj":
        print("Sujet :", token.text)

Sujet : médecin


## 3.c Son objet direct

In [10]:
for token in doc:
    if token.dep_ in ["obj", "obl:arg"]:
        print("Objet direct potentiel :", token.text)

Objet direct potentiel : antibiotiques


## 3.d Les compléments circonstanciels

In [12]:
for token in doc:
    if token.dep_ in ["obl", "nmod"]:
        cc = ' '.join([child.text for child in token.subtree])
        print("Complément circonstanciel :", cc)

Complément circonstanciel : après le diagnostic


# Étape 4 : Extraire les relations "qui fait quoi"

In [14]:
relations = []
for token in doc:
    if token.dep_ == "ROOT":
        sujet = [child.text for child in token.children if child.dep_ == "nsubj"]
        # On accepte "obj" ou "obl:arg" comme objet direct potentiel
        objet = [child.text for child in token.children if child.dep_ in ["obj", "obl:arg"]]
        if sujet and objet:
            relations.append((sujet[0], token.text, objet[0]))

print("Relations (qui fait quoi) :", relations)

Relations (qui fait quoi) : [('médecin', 'prescrit', 'antibiotiques')]


Cela signifie :

* **Le sujet** est **« médecin »** ➔ c’est celui qui fait l’action.
* **Le verbe** est **« prescrit »** ➔ c’est l’action réalisée.
* **L’objet** est **« antibiotiques »** ➔ c’est ce qui reçoit l’action.

---

### Donc, on peut reformuler la relation comme :

> **Le médecin** (sujet) **prescrit** (verbe) **des antibiotiques** (objet).

Autrement dit :

> C’est le médecin qui fait l’action de prescrire, et les antibiotiques sont ce qu’il prescrit.

# Étape 5 : Tester avec une autre langue (anglais)

## 5.1 Charger le texte avec spaCy

In [16]:
# Charger le modèle anglais
nlp_en = spacy.load("en_core_web_sm")

# Texte anglais
texte_en = "The doctor prescribes antibiotics to the feverish patient after the diagnosis."

# Traitement
doc_en = nlp_en(texte_en)

## 5.2 Afficher POS et dépendances pour chaque mot

In [17]:
# Affichage POS et dépendances
for token in doc_en:
    print(f"{token.text:<15} POS: {token.pos_:<10} Dépendance: {token.dep_}")

The             POS: DET        Dépendance: det
doctor          POS: NOUN       Dépendance: nsubj
prescribes      POS: VERB       Dépendance: ROOT
antibiotics     POS: NOUN       Dépendance: dobj
to              POS: ADP        Dépendance: prep
the             POS: DET        Dépendance: det
feverish        POS: ADJ        Dépendance: amod
patient         POS: NOUN       Dépendance: pobj
after           POS: ADP        Dépendance: prep
the             POS: DET        Dépendance: det
diagnosis       POS: NOUN       Dépendance: pobj
.               POS: PUNCT      Dépendance: punct


## 5.3 Trouver les éléments clés

### a. Le verbe principal

In [18]:
for token in doc_en:
    if token.dep_ == "ROOT":
        print("Verbe principal :", token.text)

Verbe principal : prescribes


### b. Son sujet

In [20]:
for token in doc_en:
    if token.dep_ == "nsubj":
        print("Sujet :", token.text)

Sujet : doctor


### c. Son objet direct

In [23]:
for token in doc_en:
    if token.dep_ == "dobj":  # Direct object dependency tag in spaCy
        print(f"Direct object: {token.text}")

Direct object: antibiotics


In [25]:
complements = []

for token in doc_en:
    # Identifier les prépositions (pour les compléments circonstanciels de lieu, temps, etc.)
    if token.dep_ == "prep":
        # Extraire la phrase prépositionnelle complète (ex: "to the feverish patient")
        phrase = " ".join([t.text for t in token.subtree])
        complements.append(phrase)
    # Identifier les adverbes simples (ex: "quickly", "yesterday")
    elif token.dep_ in ["advmod", "npadvmod"]:
        complements.append(token.text)

# Afficher les résultats
print("Compléments circonstanciels :")
for comp in set(complements):  # Éviter les doublons
    print(f"- {comp}")

Compléments circonstanciels :
- to the feverish patient
- after the diagnosis


## 5.4 Extraire les relations "qui fait quoi"

In [24]:
relations = []
for token in doc_en:
    if token.dep_ == "ROOT":
        sujet = [child.text for child in token.children if child.dep_ == "nsubj"]
        # On accepte "obj" ou "obl:arg" comme objet direct potentiel
        objet = [child.text for child in token.children if child.dep_ in ['dobj']]
        if sujet and objet:
            relations.append((sujet[0], token.text, objet[0]))

print("Relations (qui fait quoi) :", relations)

Relations (qui fait quoi) : [('doctor', 'prescribes', 'antibiotics')]
